In [1]:
class Point2D:
    __slots__ = ('_x', "_y")
    
    def __init__(self, x, y):
        self._x = x
        self._y = y
        
    @property
    def x(self):
        return self._x
    @property
    def y(self):
        return self._y
    
    def __eq__(self, other):
        return isinstance(other, Point) and (self.x, self.y) == (other.x, other.y)
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    def __repr__(self):
        return f"Point2D({self.x}, {self.y})"
    
    def __str__(self):
        return f"({self.x}, {self.y})"

In [2]:
class Point3D:
    __slots__ = ('_x', "_y", "_z")
    
    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z
        
    @property
    def x(self):
        return self._x
    @property
    def y(self):
        return self._y
    @property
    def z(self):
        return self._z
    
    def __eq__(self, other):
        return isinstance(other, Point) and (self.x, self.y, self.z) == (other.x, other.y, other.z)
    
    def __hash__(self):
        return hash((self.x, self.y, self.z))
    
    def __repr__(self):
        return f"Point2D({self.x}, {self.y}, {self.z})"
    
    def __str__(self):
        return f"({self.x}, {self.y}, {self.z})"

Point2D and Point3D have basically the same code. Let's use metaclass to write reusable code for such cases

We will use metaclasses instead of decorators cause we care about inheritance

we will use a field attribute to define the fields that are gonna be in the class

In [3]:
class Point2D:
    _fields = ["x", "y"]
    
    def __init__(self, x, y):
        self._x = x
        self._y = y
        
class Point3D:
    _fields = ["x", "y", "z"]
    
    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z

let's create a metaclass, that will create properties and slots and implement eq, hash, repr and str

In [26]:
class SlottedStruct(type):
    def __new__(mcls, name, bases, class_dict):
        cls_object = super().__new__(mcls, name, bases, class_dict)
        
        #__slots__
        setattr(cls_object, "__slots__", [f"_{field}" for field in cls_object._fields])
        
        #read-only property for each field
        for field in cls_object._fields:
            slot = f"_{field}"
            setattr(cls_object, field, property(fget=lambda self, attrib=slot: getattr(self, slot)))
           
            # this won't work
            # setattr(cls_object, field, property(fget=lambda self: getattr(self, slot))
        
        return cls_object

In [27]:
class Person(metaclass=SlottedStruct):
    _fields= ['name', 'age']
    
    def __init__(self, name, age):
        self._name = name
        self._age = age
    

In [28]:
vars(Person)

mappingproxy({'__module__': '__main__',
              '_fields': ['name', 'age'],
              '__init__': <function __main__.Person.__init__(self, name, age)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              '__slots__': ['_name', '_age'],
              'name': <property at 0x15e9ace9bd8>,
              'age': <property at 0x15e9ace99a8>})

In [29]:
p = Person("Alex", 19)

In [30]:
p.name, p.age

(19, 19)

In [31]:
p.name = "Alexa"

AttributeError: can't set attribute

let's add some more

In [52]:
class SlottedStruct(type):
    def __new__(mcls, name, bases, class_dict):
        cls_object = super().__new__(mcls, name, bases, class_dict)
        
        #__slots__
        setattr(cls_object, "__slots__", [f"_{field}" for field in cls_object._fields])
        
        #read-only property for each field
        for field in cls_object._fields:
            slot = f"_{field}"
            setattr(cls_object, field, property(fget=lambda self, attrib=slot: getattr(self, attrib)))
        
        #__eq__
        def eq(self, other):
            if isinstance(other, cls_object):
                self_fields = [getattr(self, field) for field in cls_object._fields]
                other_fields = [getattr(other, field) for field in cls_object._fields]
                return self_fields == other_fields
            return False
        setattr(cls_object, "__eq__", eq)
        
        #__hash__ we make an assumption that all the field values are hashable
        def hash_(self):
            field_values = (getattr(self, field) for field in cls_object._fields)
            return hash(tuple(field_values))
        setattr(cls_object, "__hash__", hash_)
        
        #__str__
        def str_(self):
            field_values = (getattr(self, field) for field in cls_object._fields) 
            field_values_joined = ', '.join(map(str, field_values))
            return f"{cls_object.__name__}({field_values_joined})"
        setattr(cls_object, "__str__", str_)
        
        #__repr__
#         def repr_(self): # works too
#             field_values = (getattr(self, field) for field in cls_object._fields)
#             field_key_values = (f'{key}={value}' for key, value in zip(cls_object._fields, field_values))
#             field_key_values_str = ', '.join(field_key_values)
#             return f"{cls_object.__name__}({field_key_values_str})"
        def repr_(self):
            field_values = (getattr(self, field) for field in self._fields)
            field_key_values = (f'{key}={value}' for key, value in zip(cls_object._fields, field_values))
            field_key_values_str = ', '.join(field_key_values)
            return f"{self.__class__.__name__}({field_key_values_str})"
        setattr(cls_object, "__repr__", repr_)
        
        return cls_object

In [53]:
class Person(metaclass=SlottedStruct):
    _fields= ['name', 'age']
    
    def __init__(self, name, age):
        self._name = name
        self._age = age
    

In [54]:
type(Person)

__main__.SlottedStruct

In [55]:
p1 = Person("Alex", 19)
p2 = Person("Alex", 19)
p3 = Person("Oleg", 28)

In [56]:
p1 == p2, p1 == p3

(True, False)

In [57]:
hash(p1), hash(p2), hash(p3),

(7806736872228484413, 7806736872228484413, -163906605747773004)

In [59]:
str(p1), str(p2), str(p3),

('Person(Alex, 19)', 'Person(Alex, 19)', 'Person(Oleg, 28)')

In [60]:
repr(p1), repr(p2), repr(p3),

('Person(name=Alex, age=19)',
 'Person(name=Alex, age=19)',
 'Person(name=Oleg, age=28)')

now we can make our Point classes

In [63]:
class Point2D(metaclass=SlottedStruct):
    _fields = ("x", "y")
    
    def __init__(self, x, y):
        self._x = x
        self._y = y
        
class Point3D(metaclass=SlottedStruct):
    _fields = ('x', "y", "z")
    
    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z

In [65]:
p1 = Point2D(1, 2)
p2 = Point2D(1, 2)
p3 = Point2D(0, 0)

In [66]:
p1 == p2

True

let's create a special class decorator so we won't have to write metaclass=SlottedStruct every time

class have properties `__name__`, `__dict__`, `__bases__`,

In [68]:
Point2D.__dict__, 

(mappingproxy({'__module__': '__main__',
               '_fields': ('x', 'y'),
               '__init__': <function __main__.Point2D.__init__(self, x, y)>,
               '__dict__': <attribute '__dict__' of 'Point2D' objects>,
               '__weakref__': <attribute '__weakref__' of 'Point2D' objects>,
               '__doc__': None,
               '__slots__': ['_x', '_y'],
               'x': <property at 0x15e9acf79a8>,
               'y': <property at 0x15e9acf7d68>,
               '__eq__': <function __main__.SlottedStruct.__new__.<locals>.eq(self, other)>,
               '__hash__': <function __main__.SlottedStruct.__new__.<locals>.hash_(self)>,
               '__str__': <function __main__.SlottedStruct.__new__.<locals>.str_(self)>,
               '__repr__': <function __main__.SlottedStruct.__new__.<locals>.repr_(self)>}),)

In [69]:
Point2D.__name__,  Point2D.__bases__, 

('Point2D', (object,))

In [73]:
def struct(cls):
    return SlottedStruct(cls.__name__, cls.__bases__, dict(cls.__dict__))

In [74]:
@struct
class Point2D:
    _fields = ("x", "y")
    
    def __init__(self, x, y):
        self._x = x
        self._y = y
@struct        
class Point3D:
    _fields = ('x', "y", "z")
    
    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z

In [75]:
p1 = Point2D(1, 2)
p2 = Point2D(1, 2)
p3 = Point2D(0, 0)

In [76]:
type(Point2D)

__main__.SlottedStruct

In [78]:
str(p1), repr(p1)

('Point2D(1, 2)', 'Point2D(x=1, y=2)')

in python 3.7 we have data classes and don't need to do all this stuff with metaclasses

### why

#### this won't work

            setattr(cls_object, field, property(fget=lambda self: getattr(self, slot))

In [15]:
class Obj:
    _fields = ["a", "b", "c"]
    def __init__(self, a, b, c):
        self._a = a
        self._b = b
        self._c = c

In [16]:
for field in Obj._fields:
    slot = f"_{field}"
    setattr(Obj, field, property(fget=lambda self: getattr(self, slot)))

In [17]:
obj = Obj(1, 2, 3)

In [18]:
obj.a

3

In [19]:
obj.b

3

In [20]:
obj.c

3

In [21]:
Obj.__dict__

mappingproxy({'__module__': '__main__',
              '_fields': ['a', 'b', 'c'],
              '__init__': <function __main__.Obj.__init__(self, a, b, c)>,
              '__dict__': <attribute '__dict__' of 'Obj' objects>,
              '__weakref__': <attribute '__weakref__' of 'Obj' objects>,
              '__doc__': None,
              'a': <property at 0x15e9acdbe08>,
              'b': <property at 0x15e9ace7098>,
              'c': <property at 0x15e9ace70e8>})

In [22]:
vars(obj)

{'_a': 1, '_b': 2, '_c': 3}

what's happening:

property(fget=lambda self: getattr(self, slot)) is closure

it's using `slot` that coming outside its scope inside function `__new__`

what's happening with value for slot inside lambda:

slot from getattr(self, slot) is pointing to slot = f"_{field}", but we are in a loop. So every lambda is pointing to the same slot, the last one in a loop